In [2]:
import pandas as pd
import plotly.express as px

In [11]:
model = "EFFNETV2S"
dset = "PlantVillage"
model_set = model + "_" + dset

In [12]:
# classes = ['Baked Potato', 'Burger', 'Crispy Chicken', 'Donut', 'Fries', 'Hot Dog', 'Pizza', 'Sandwich', 'Taco', 'Taquito']
# cnts = [1500, 1164, 1404,  513, 1500,  924,  136, 1301, 1500, 1198]

In [40]:
base = pd.read_csv(f"../logs/{model_set}_base.csv")
base_pc = pd.read_csv(f"../logs/{model_set}_base_perclass.csv")
df = pd.read_csv(f"../logs/{model_set}.csv")
df_pc = pd.read_csv(f"../logs/{model_set}_perclass.csv")

In [51]:
base.loc[base.epoch==14].mean()

epoch          14.00000
train acc       0.91380
train loss      0.00446
train f1        0.91384
val acc         0.92600
val loss        1.66116
val f1          0.92604
test acc        0.93040
test loss       0.38200
test f1         0.93042
weighted f1     0.93016
experiment      2.00000
dtype: float64

In [42]:
measures = ["test acc", "test loss", "test f1", "weighted f1"]
measure_labels = ["Dokładność [%]", "Strata", "Dokładność F1 (globalna)", "Dokładność F1 (średnia ważona)"]
heuristic_names = {"least_confidence": "Najmniejsza Pewność" , "smallest_margin":"Najmniejszy Margines", "largest_margin": "Największy Margines", "representative_sampling":"Próbkowanie Reprezentatywne (PR)", 
                   "mc_dropout":"Wyłączenie Monte Carlo (MC)", "representative_mc_dropout":"PR + MC", "None":"Wybór Losowy"}

In [43]:
base_melted = pd.melt(base.groupby("epoch").mean()[["val f1", "val loss", "test f1", "test loss"]].reset_index(), value_vars=["val f1", "test f1", "val loss", "test loss"], id_vars="epoch")

In [44]:
base_melted

,epoch,variable,value
0,0,val f1,0.83748
1,1,val f1,0.87246
2,2,val f1,0.89116
3,3,val f1,0.90160
4,4,val f1,0.90512
...,...,...,...
115,25,test loss,0.47034
116,26,test loss,0.86122
117,27,test loss,0.29178
118,28,test loss,0.76804


In [45]:
legend_labels = {"val f1": "Walidacyjny", "test f1":"Testowy", "val loss": "Walidacyjny", "test loss":"Testowy"}
for m, l in zip(("f1", "loss"),("Dokładność F1", "Strata")):
    fig = px.line(
            base_melted.loc[base_melted.variable.str.contains(m)],
            x = "epoch",
            y = "value",
            color = "variable",
            color_discrete_sequence = px.colors.qualitative.Bold,
            markers=True,
            width=800,
            height = 600,
            render_mode="svg",
            labels = {"variable": "Zbiór", "epoch":"Epoka", "value": l},
        )
    fig.for_each_trace(lambda t: t.update(name = legend_labels[t.name],
                                        legendgroup = legend_labels[t.name],
                                        hovertemplate = t.hovertemplate.replace(t.name, legend_labels[t.name])
                                        )
                    )
    fig.update_layout(
        plot_bgcolor='white',
    )
    fig.update_xaxes(
        mirror=True,
        ticks='outside',
        showline=True,
        linecolor='black',
        gridcolor='lightgrey'
    )
    fig.update_yaxes(
        mirror=True,
        ticks='outside',
        showline=True,
        linecolor='black',
        gridcolor='lightgrey'
    )

    if "loss" in m:
        fig.update_layout(legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="right",
            x=0.99 ,
            bordercolor="Black",
            borderwidth=1))
    else:
        fig.update_layout(legend=dict(
            yanchor="bottom",
            y=0.01,
            xanchor="right",
            x=0.99 ,
            bordercolor="Black",
            borderwidth=1
        ))
        # fig.show()
    fig.write_image(f"../imgs/{model_set}_{m.replace(' ','_')}_val_test.png")
    # fig.show()

In [46]:
grouped_base = base.loc[base.epoch==14, measures].mean()
grouped_base["test acc"] = grouped_base["test acc"]*100

In [47]:
grouped_df = df.loc[df.epoch == 14].groupby(["active_iter", "heuristic"])[measures].mean()
grouped_df["test acc"] = grouped_df["test acc"]*100

In [48]:
for m,l in zip(measures, measure_labels):
    fig = px.line(
        grouped_df.reset_index(),
        x = "active_iter",
        y = m,
        color = "heuristic",
        color_discrete_sequence = px.colors.qualitative.Bold,
        markers=True,
        width=800,
        height = 600,
        category_orders={
            "heuristic":["least_confidence", "smallest_margin", "largest_margin", "representative_sampling", "mc_dropout", "representative_mc_dropout", None]
        },
        labels = {"heuristic": "Heurystyka", "active_iter":"Iteracja", m:l},
        render_mode="svg"
    )
    fig.for_each_trace(lambda t: t.update(name = heuristic_names[t.name],
                                      legendgroup = heuristic_names[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, heuristic_names[t.name])
                                     )
                  )
    fig.add_hline(y=grouped_base[m], line_dash="dash", line_width=1, annotation_text="Wartość bazowa", 
              annotation_position="bottom left")
    fig.update_layout(
        plot_bgcolor='white',
    )
    fig.update_xaxes(
        mirror=True,
        ticks='outside',
        showline=True,
        linecolor='black',
        gridcolor='lightgrey'
    )
    fig.update_yaxes(
        mirror=True,
        ticks='outside',
        showline=True,
        linecolor='black',
        gridcolor='lightgrey'
    )
    if "loss" in m:
        fig.update_layout(legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="right",
            x=0.99 ,
            bordercolor="Black",
            borderwidth=1))
    else:
        fig.update_layout(legend=dict(
            yanchor="bottom",
            y=0.01,
            xanchor="right",
            x=0.99 ,
            bordercolor="Black",
            borderwidth=1
        ))
    # fig.show()
    fig.write_image(f"../imgs/{model_set}_{m.replace(' ','_')}.png")

In [49]:
grouped_df_pc = df_pc.loc[df_pc.epoch == 14].groupby(["active_iter", "heuristic", "class"])[["test acc", "test f1",]].mean()
grouped_df_pc["test acc"] = grouped_df_pc["test acc"]*100

In [50]:
grouped_base_pc = base_pc.loc[base_pc.epoch == 14].groupby(["class"])[["test acc", "test f1",]].mean()
grouped_base_pc["test acc"] = grouped_base_pc["test acc"]*100

In [38]:
grouped_df_pc.reset_index()["class"].unique()

array([], dtype=float64)

In [39]:
for c, cnt in zip(grouped_df_pc.reset_index()["class"].unique(), [0 for i in range(15)]):
    print(c)
    for m,l in zip(["test acc", "test f1",],["Dokładność [%]", "Dokładność F1"]):
        fig = px.line(
            grouped_df_pc.reset_index().loc[grouped_df_pc.reset_index()["class"]==c].reset_index(),
            x = "active_iter",
            y = m,
            color = "heuristic",
            color_discrete_sequence = px.colors.qualitative.Bold,
            markers=True,
            width=800,
            height = 600,
            category_orders={
                "heuristic":["least_confidence", "smallest_margin", "largest_margin", "representative_sampling", "mc_dropout", "representative_mc_dropout", None]
            },
            labels = {"heuristic": "Heurystyka", "active_iter":"Iteracja", m:l},
            render_mode="svg",
            title = f"{c} ({cnt})"
        )
        fig.for_each_trace(lambda t: t.update(name = heuristic_names[t.name],
                                        legendgroup = heuristic_names[t.name],
                                        hovertemplate = t.hovertemplate.replace(t.name, heuristic_names[t.name])
                                        ))
        fig.add_hline(y=grouped_base_pc.loc[c,m], line_dash="dash", line_width=1, annotation_text="Wartość bazowa", 
                annotation_position="bottom left")
        fig.update_layout(
            plot_bgcolor='white',
            title=dict(
                yanchor="top",
                y=0.9,
                xanchor="left",
                x=0.1 ,
            )
        )
        fig.update_xaxes(
            mirror=True,
            ticks='outside',
            showline=True,
            linecolor='black',
            gridcolor='lightgrey'
        )
        fig.update_yaxes(
            mirror=True,
            ticks='outside',
            showline=True,
            linecolor='black',
            gridcolor='lightgrey'
        )
        if "loss" in m:
            fig.update_layout(legend=dict(
                yanchor="top",
                y=0.99,
                xanchor="right",
                x=0.99 ,
                bordercolor="Black",
                borderwidth=1))
        else:
            fig.update_layout(legend=dict(
                yanchor="bottom",
                y=0.01,
                xanchor="right",
                x=0.99 ,
                bordercolor="Black",
                borderwidth=1
            ))
        # fig.show()
        fig.write_image(f"../imgs/{model_set}_{m.replace(' ','_')}_{c.replace(' ','_')}.png")

In [8]:
df = pd.read_csv("../logs/EFFNETV2S_PlantVillage.csv")

In [9]:
grouped_df = df.loc[df.epoch == 14].groupby(["active_iter", "heuristic"])[measures].mean()
grouped_df["test acc"] = grouped_df["test acc"]*100

In [10]:
for m,l in zip(measures, measure_labels):
    fig = px.line(
        grouped_df.reset_index(),
        x = "active_iter",
        y = m,
        color = "heuristic",
        color_discrete_sequence = px.colors.qualitative.Bold,
        markers=True,
        width=800,
        height = 600,
        category_orders={
            "heuristic":["least_confidence", "smallest_margin", "largest_margin", "representative_sampling", "mc_dropout", "representative_mc_dropout", None]
        },
        labels = {"heuristic": "Heurystyka", "active_iter":"Iteracja", m:l},
        render_mode="svg"
    )
    fig.for_each_trace(lambda t: t.update(name = heuristic_names[t.name],
                                      legendgroup = heuristic_names[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, heuristic_names[t.name])
                                     )
                  )
    # fig.add_hline(y=grouped_base[m], line_dash="dash", line_width=1, annotation_text="Wartość bazowa", 
    #           annotation_position="bottom left")
    fig.update_layout(
        plot_bgcolor='white',
    )
    fig.update_xaxes(
        mirror=True,
        ticks='outside',
        showline=True,
        linecolor='black',
        gridcolor='lightgrey'
    )
    fig.update_yaxes(
        mirror=True,
        ticks='outside',
        showline=True,
        linecolor='black',
        gridcolor='lightgrey'
    )
    if "loss" in m:
        fig.update_layout(legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="right",
            x=0.99 ,
            bordercolor="Black",
            borderwidth=1))
    else:
        fig.update_layout(legend=dict(
            yanchor="bottom",
            y=0.01,
            xanchor="right",
            x=0.99 ,
            bordercolor="Black",
            borderwidth=1
        ))
    fig.show()
    fig.write_image(f"../imgs/{model_set}_{m.replace(' ','_')}.png")